<a href="https://colab.research.google.com/github/sanjeev1996/Data-Extraction-_-Text-Analysis/blob/master/Tensorflow2_0_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu==2.0.0

In [1]:
import tensorflow as tf
import numpy as np
rng = np.random
print(tf.__version__)

2.0.0


In [0]:
num_classes = 10 # 0 to 9 digits 
num_features = 784 # 28*28

# Training dataset parameters
learning_rate = 0.01
training_steps = 1000
batch_size = 250
display_step = 50

In [3]:
from tensorflow.keras.datasets import mnist
# Prepare MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_train, np.float32)
#print(x_test.shape)

# Flatten image to 1-D vector of 784 features
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize image from [0,255] to [0,1]
x_train, x_test = x_train/255., x_test/255.

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [0]:
# Weight of shape (784,10), the 28*28 image features, and total number of classe
W = tf.Variable(tf.ones([num_features, num_classes]), name='weight')
# Bias of hape[10], the total number of classes
b = tf.Variable(tf.zeros([num_classes]), name='bias')

# Logistic regression (Wx + b) 
def logistic_regression(x):
  # Apply softmax to normalize the logits to a probability distribution.
  return tf.nn.softmax(tf.matmul(x,W)+b)

# loss function
def cross_entropy(y_pred, y_true):
  # Encode label to a one hot vector
  y_true = tf.one_hot(y_true, depth=num_classes)
  # clip predication values to avoid log(0) error
  y_pred =tf.clip_by_value(y_pred, 1e-9,1.)
  return tf.reduce_mean(-tf.reduce_sum(y_true*tf.math.log(y_pred)))

def accuracy(y_pred, y_true):
  # highest score in predicton vector
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.optimizers.SGD(learning_rate)

In [0]:
def run_optimization(x, y):
  # computation insider a gradientTape for automatic differentiation
  with tf.GradientTape() as g:
    pred = logistic_resgression(x)
    loss = cross_entropy(pred, y)
  
  gradients = g.gradient(loss, [W, b])
  optimizer.apply_gradients(zip(gradients, [W, b]))

In [24]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
  run_optimization(batch_x, batch_y)
  
  if step % display_step == 0:
    pred = logistic_regression(batch_x)
    loss = cross_entropy(pred, batch_y)
    acc = accuracy(pred, batch_y)
    print("step: %i, loss: %f, accuracy: %f," %(step, loss, acc))

step: 50, loss: 1249.969482, accuracy: 0.708000,
step: 100, loss: 1058.296631, accuracy: 0.724000,
step: 150, loss: 1019.301025, accuracy: 0.748000,
step: 200, loss: 595.635742, accuracy: 0.752000,
step: 250, loss: 534.047302, accuracy: 0.844000,
step: 300, loss: 577.187317, accuracy: 0.848000,
step: 350, loss: 489.918823, accuracy: 0.860000,
step: 400, loss: 560.826843, accuracy: 0.784000,
step: 450, loss: 590.054626, accuracy: 0.816000,
step: 500, loss: 601.623596, accuracy: 0.836000,
step: 550, loss: 490.200653, accuracy: 0.832000,
step: 600, loss: 455.398224, accuracy: 0.840000,
step: 650, loss: 697.726562, accuracy: 0.828000,
step: 700, loss: 539.690613, accuracy: 0.848000,
step: 750, loss: 644.535095, accuracy: 0.840000,
step: 800, loss: 468.825378, accuracy: 0.828000,
step: 850, loss: 609.608765, accuracy: 0.848000,
step: 900, loss: 449.452026, accuracy: 0.872000,
step: 950, loss: 588.423401, accuracy: 0.852000,
step: 1000, loss: 516.873230, accuracy: 0.796000,


In [25]:
# predict image
import matplotlib.pyplot as plt
n_images = 1
test_images = x_test[5]
predictions = logistic_regression(test_images)

# Display image and model prediction
for i in range(n_image):
  plt.imshow(np.reshape(test_images[i], [28,28], cmap='gray'))
  plt.show()
  print("Model prediction   ")

InvalidArgumentError: ignored